In [1]:
# Import packages
import os
from google.colab import drive

# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.4.0'
spark_version = 'spark-3.4.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

# Import Remaining Packages (Post-Install)
from pyspark.sql import SparkSession
from pyspark import SparkFiles
from pyspark.ml.feature import VectorAssembler, StandardScaler
from pyspark.sql import functions as F
from pyspark.sql.functions import round, col
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:6 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Get:7 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [43.3 kB]
Get:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Get:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Get:10 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [850 kB]
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 http://archive.ubuntu.com/ubuntu ja

In [2]:
# Create a SparkSession
spark = SparkSession.builder.appName("SparkML").getOrCreate()
spark

In [3]:
# Connect to google drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [4]:
# Read in the csv data from google drive
# Double check to make sure your pathway to the csv's is the same!!!!
spark.sparkContext.addFile('/content/gdrive/MyDrive/Bootcamp/Project4/Data/train_timeseries.csv')
train_timeseries_df = spark.read.csv(SparkFiles.get('/content/gdrive/MyDrive/Bootcamp/Project4/Data/train_timeseries.csv'), sep=",", header=True, inferSchema=True)

spark.sparkContext.addFile('/content/gdrive/MyDrive/Bootcamp/Project4/Data/test_timeseries.csv')
test_timeseries_df = spark.read.csv(SparkFiles.get('/content/gdrive/MyDrive/Bootcamp/Project4/Data/test_timeseries.csv'), sep=",", header=True, inferSchema=True)

spark.sparkContext.addFile('/content/gdrive/MyDrive/Bootcamp/Project4/Data/soil_data.csv')
soil_data_df = spark.read.csv(SparkFiles.get('/content/gdrive/MyDrive/Bootcamp/Project4/Data/soil_data.csv'), sep=",", header=True, inferSchema=True)

In [5]:
# Preview soil data
soil_data_df.show()

+----+---------+----------+---------+------+------+------+------+------+------+------+------+-------+-------+-------+-------+-------------+-----------------+----------------+-----------------+-----------------+----------------+-----------------+-------------------+-------------------+---+---+---+---+---+---+---+
|fips|      lat|       lon|elevation|slope1|slope2|slope3|slope4|slope5|slope6|slope7|slope8|aspectN|aspectE|aspectS|aspectW|aspectUnknown|         WAT_LAND|        NVG_LAND|         URB_LAND|         GRS_LAND|        FOR_LAND|      CULTRF_LAND|        CULTIR_LAND|          CULT_LAND|SQ1|SQ2|SQ3|SQ4|SQ5|SQ6|SQ7|
+----+---------+----------+---------+------+------+------+------+------+------+------+------+-------+-------+-------+-------+-------------+-----------------+----------------+-----------------+-----------------+----------------+-----------------+-------------------+-------------------+---+---+---+---+---+---+---+
|1001|32.536382| -86.64449|       63|0.0419|0.2788|0.2984|

In [6]:
# Preview training time series data
train_timeseries_df.show()

+----+----------+-------+------+-----+-----+------+------+-------+-------+---------+-----+-----+---------+---------+-----------+-----+---------+---------+-----------+-----+
|fips|      date|PRECTOT|    PS| QV2M|  T2M|T2MDEW|T2MWET|T2M_MAX|T2M_MIN|T2M_RANGE|   TS|WS10M|WS10M_MAX|WS10M_MIN|WS10M_RANGE|WS50M|WS50M_MAX|WS50M_MIN|WS50M_RANGE|score|
+----+----------+-------+------+-----+-----+------+------+-------+-------+---------+-----+-----+---------+---------+-----------+-----+---------+---------+-----------+-----+
|1001|2000-01-01|   0.22|100.51| 9.65|14.74| 13.51| 13.51|  20.96|  11.46|      9.5|14.65|  2.2|     2.94|     1.49|       1.46| 4.85|     6.04|     3.23|       2.81| null|
|1001|2000-01-02|    0.2|100.55|10.42|16.69| 14.71| 14.71|   22.8|  12.61|    10.18| 16.6| 2.52|     3.43|     1.83|        1.6| 5.33|     6.13|     3.72|       2.41| null|
|1001|2000-01-03|   3.65|100.15|11.76|18.49| 16.52| 16.52|  22.73|  15.32|     7.41|18.41| 4.03|     5.33|     2.66|       2.67| 7.53| 

In [7]:
# Initial count of score column
train_timeseries_df.groupBy('score').count().orderBy(F.desc('count')).show()

+--------------------+--------+
|               score|   count|
+--------------------+--------+
|                null|16543884|
|                 0.0| 1480827|
|                 1.0|  219128|
|                 2.0|  123789|
|                 3.0|   82801|
|                 4.0|   45841|
|                 5.0|   21806|
|              1.0E-4|    1037|
|              2.0E-4|     742|
|  0.9998999999999999|     681|
|              3.0E-4|     642|
|  1.9998999999999998|     487|
|              4.0E-4|     486|
|              5.0E-4|     478|
|              6.0E-4|     459|
|              1.0001|     455|
|              0.9998|     449|
|7.000000000000001E-4|     409|
|  0.9995999999999999|     366|
|              0.9997|     353|
+--------------------+--------+
only showing top 20 rows



In [8]:
# Drop NA in score column
train_timeseries_df = train_timeseries_df.na.drop(subset=["score"])

# Round score column
train_timeseries_df = train_timeseries_df.withColumn('score', round(train_timeseries_df['score']))

# Cast score column to integer
train_timeseries_df = train_timeseries_df.withColumn('score', col('score').cast('integer'))

# Show adjusted score column
train_timeseries_df.groupBy('score').count().orderBy(F.desc('count')).show()

+-----+-------+
|score|  count|
+-----+-------+
|    0|1652209|
|    1| 466965|
|    2| 295329|
|    3| 196804|
|    4| 106264|
|    5|  39225|
+-----+-------+



In [9]:
# Review datatypes
train_timeseries_df.printSchema()

root
 |-- fips: integer (nullable = true)
 |-- date: date (nullable = true)
 |-- PRECTOT: double (nullable = true)
 |-- PS: double (nullable = true)
 |-- QV2M: double (nullable = true)
 |-- T2M: double (nullable = true)
 |-- T2MDEW: double (nullable = true)
 |-- T2MWET: double (nullable = true)
 |-- T2M_MAX: double (nullable = true)
 |-- T2M_MIN: double (nullable = true)
 |-- T2M_RANGE: double (nullable = true)
 |-- TS: double (nullable = true)
 |-- WS10M: double (nullable = true)
 |-- WS10M_MAX: double (nullable = true)
 |-- WS10M_MIN: double (nullable = true)
 |-- WS10M_RANGE: double (nullable = true)
 |-- WS50M: double (nullable = true)
 |-- WS50M_MAX: double (nullable = true)
 |-- WS50M_MIN: double (nullable = true)
 |-- WS50M_RANGE: double (nullable = true)
 |-- score: integer (nullable = true)



In [10]:
# Columns that we do not want to include as features
exclude_columns = ['score', 'fips', 'date']

# Set feature columns (excluding non-feature columns)
feature_columns = [col_name for col_name in train_timeseries_df.columns if col_name not in exclude_columns]
feature_columns

['PRECTOT',
 'PS',
 'QV2M',
 'T2M',
 'T2MDEW',
 'T2MWET',
 'T2M_MAX',
 'T2M_MIN',
 'T2M_RANGE',
 'TS',
 'WS10M',
 'WS10M_MAX',
 'WS10M_MIN',
 'WS10M_RANGE',
 'WS50M',
 'WS50M_MAX',
 'WS50M_MIN',
 'WS50M_RANGE']

In [11]:
# Create VectorAssembler instance to combine feature columns into a single dense vector column
assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")
assembler

VectorAssembler_001fb7b710b9

In [12]:
# Apply VectorAssembler to create the dense vector column
df_with_features = assembler.transform(train_timeseries_df)
df_with_features.show()

+----+----------+-------+------+-----+-----+------+------+-------+-------+---------+-----+-----+---------+---------+-----------+-----+---------+---------+-----------+-----+--------------------+
|fips|      date|PRECTOT|    PS| QV2M|  T2M|T2MDEW|T2MWET|T2M_MAX|T2M_MIN|T2M_RANGE|   TS|WS10M|WS10M_MAX|WS10M_MIN|WS10M_RANGE|WS50M|WS50M_MAX|WS50M_MIN|WS50M_RANGE|score|            features|
+----+----------+-------+------+-----+-----+------+------+-------+-------+---------+-----+-----+---------+---------+-----------+-----+---------+---------+-----------+-----+--------------------+
|1001|2000-01-04|  15.95|100.29| 6.42| 11.4|  6.09|   6.1|  18.09|   2.16|    15.92|11.31| 3.84|     5.67|     2.08|       3.59| 6.73|     9.31|     3.74|       5.58|    1|[15.95,100.29,6.4...|
|1001|2000-01-11|   1.33| 100.4| 6.63|11.48|  7.84|  7.84|  18.88|   5.72|    13.16|10.43| 1.76|     2.48|     1.05|       1.43| 3.55|     6.38|     1.71|       4.67|    2|[1.33,100.4,6.63,...|
|1001|2000-01-18|   1.11|100.3

In [13]:
# Apply feature scaling using StandardScaler
scaler = StandardScaler(inputCol="features", outputCol="scaled_features", withMean=True, withStd=True)
scaler

StandardScaler_8ee23129d766

In [14]:
# Fit the StandardScaler to compute mean and standard deviation
scaler_model = scaler.fit(df_with_features)
scaler_model

StandardScalerModel: uid=StandardScaler_8ee23129d766, numFeatures=18, withMean=true, withStd=true

In [15]:
# Transform the DataFrame to get scaled feature vectors
df_with_scaled_features = scaler_model.transform(df_with_features)
df_with_scaled_features.show()

+----+----------+-------+------+-----+-----+------+------+-------+-------+---------+-----+-----+---------+---------+-----------+-----+---------+---------+-----------+-----+--------------------+--------------------+
|fips|      date|PRECTOT|    PS| QV2M|  T2M|T2MDEW|T2MWET|T2M_MAX|T2M_MIN|T2M_RANGE|   TS|WS10M|WS10M_MAX|WS10M_MIN|WS10M_RANGE|WS50M|WS50M_MAX|WS50M_MIN|WS50M_RANGE|score|            features|     scaled_features|
+----+----------+-------+------+-----+-----+------+------+-------+-------+---------+-----+-----+---------+---------+-----------+-----+---------+---------+-----------+-----+--------------------+--------------------+
|1001|2000-01-04|  15.95|100.29| 6.42| 11.4|  6.09|   6.1|  18.09|   2.16|    15.92|11.31| 3.84|     5.67|     2.08|       3.59| 6.73|     9.31|     3.74|       5.58|    1|[15.95,100.29,6.4...|[2.11848627313622...|
|1001|2000-01-11|   1.33| 100.4| 6.63|11.48|  7.84|  7.84|  18.88|   5.72|    13.16|10.43| 1.76|     2.48|     1.05|       1.43| 3.55|     6

In [16]:
# PREPARE TEST DATA

# Drop NA in score column
test_timeseries_df = test_timeseries_df.na.drop(subset=["score"])

# Round score column
test_timeseries_df = test_timeseries_df.withColumn('score', round(test_timeseries_df['score']))

# Cast score column to integer
test_timeseries_df = test_timeseries_df.withColumn('score', col('score').cast('integer'))

# Show adjusted score column
test_timeseries_df.groupBy('score').count().orderBy(F.desc('count')).show()

# Create VectorAssembler instance to combine feature columns into a single dense vector column
assembler_test = VectorAssembler(inputCols=feature_columns, outputCol="features")

# Apply VectorAssembler to create the dense vector column
test_df_with_features = assembler_test.transform(test_timeseries_df)

# Apply feature scaling using StandardScaler
test_scaler = StandardScaler(inputCol="features", outputCol="scaled_features", withMean=True, withStd=True)

# Fit the StandardScaler to compute mean and standard deviation
test_scaler_model = test_scaler.fit(test_df_with_features)

# Transform the DataFrame to get scaled feature vectors
test_df_with_scaled_features = test_scaler_model.transform(test_df_with_features)
test_df_with_scaled_features.show()

+-----+------+
|score| count|
+-----+------+
|    0|247720|
|    1| 42486|
|    2| 22159|
|    3|  9590|
|    4|  3578|
|    5|   807|
+-----+------+

+----+----------+-------+------+-----+-----+------+------+-------+-------+---------+-----+-----+---------+---------+-----------+-----+---------+---------+-----------+-----+--------------------+--------------------+
|fips|      date|PRECTOT|    PS| QV2M|  T2M|T2MDEW|T2MWET|T2M_MAX|T2M_MIN|T2M_RANGE|   TS|WS10M|WS10M_MAX|WS10M_MIN|WS10M_RANGE|WS50M|WS50M_MAX|WS50M_MIN|WS50M_RANGE|score|            features|     scaled_features|
+----+----------+-------+------+-----+-----+------+------+-------+-------+---------+-----+-----+---------+---------+-----------+-----+---------+---------+-----------+-----+--------------------+--------------------+
|1001|2019-01-01|   2.25|100.51| 9.69|14.71| 13.55| 13.52|  17.38|  10.92|     6.46|14.63|  1.2|      1.5|     0.79|       0.71| 2.74|     4.01|     1.23|       2.78|    0|[2.25,100.51,9.69...|[-0.0834925

In [17]:
# Define the model
rf = RandomForestClassifier(featuresCol='scaled_features', labelCol='score')
rf

RandomForestClassifier_3945bc8f45e4

In [18]:
# Train the model
rf_model = rf.fit(df_with_scaled_features)
rf_model

RandomForestClassificationModel: uid=RandomForestClassifier_3945bc8f45e4, numTrees=20, numClasses=6, numFeatures=18

In [19]:
# Make predictions
predictions = rf_model.transform(test_df_with_scaled_features)
predictions

DataFrame[fips: int, date: date, PRECTOT: double, PS: double, QV2M: double, T2M: double, T2MDEW: double, T2MWET: double, T2M_MAX: double, T2M_MIN: double, T2M_RANGE: double, TS: double, WS10M: double, WS10M_MAX: double, WS10M_MIN: double, WS10M_RANGE: double, WS50M: double, WS50M_MAX: double, WS50M_MIN: double, WS50M_RANGE: double, score: int, features: vector, scaled_features: vector, rawPrediction: vector, probability: vector, prediction: double]

In [20]:
# Evaluate the model
evaluator = MulticlassClassificationEvaluator(labelCol="score", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Test set accuracy = " + str(accuracy))

Test set accuracy = 0.7590856162284734


In [21]:
# Get feature importance
importances = rf_model.featureImportances
for i in range(len(feature_columns)):
    print(f"Importance of feature {feature_columns[i]}: {importances[i]}")

Importance of feature PRECTOT: 0.02393133981054897
Importance of feature PS: 0.22410796588629708
Importance of feature QV2M: 0.04032705773102391
Importance of feature T2M: 0.09925200784951221
Importance of feature T2MDEW: 0.02695971018692035
Importance of feature T2MWET: 0.018289601649166758
Importance of feature T2M_MAX: 0.2671125308908474
Importance of feature T2M_MIN: 0.0027053454561875904
Importance of feature T2M_RANGE: 0.13774979316465383
Importance of feature TS: 0.1488162902104497
Importance of feature WS10M: 0.00018410829219831372
Importance of feature WS10M_MAX: 0.000622954668455873
Importance of feature WS10M_MIN: 0.0
Importance of feature WS10M_RANGE: 0.009524886096630906
Importance of feature WS50M: 5.3818632006306046e-05
Importance of feature WS50M_MAX: 0.00036258947510059533
Importance of feature WS50M_MIN: 0.0
Importance of feature WS50M_RANGE: 0.0


In [22]:
# Save the model
model_path = "/content/gdrive/MyDrive/Bootcamp/Project4/Trained_Models/rf_model"
rf_model.save(model_path)